In [1]:
import pandas as pd

# Predicting Authorship of the Disputed Federalist Papers

The Federalist Papers are a collection of **85 essays** written by James Madison, Alexander Hamilton, and John Jay under the collective pseudonym "Publius" to promote the ratification of the United States Constitution.

Authorship of most of the papers were revealed some years later by Hamilton, though his claim to authorshipt of 12 papers were disputed for nearly 200 years (studies generally agree that the disputed essays were written by James Madison.)

| Author | Papers |
| :- | -: | 
| Jay | 2, 3, 4, 5, 64
| Madison | 10, 14, 37-48
| Hamilton | 1, 6, 7, 8, 9, 11, 12, 13, 15, 16, 17, 21-36, 59, 60, 61, 65-85
| Hamilton and Madison | 18, 19, 20
| Disputed | 49-58, 62, 63

The goal of this problem is to train a classifier that predicts the author of the disputed papers.

In [2]:
# load Federalist papers data
url = 'https://raw.githubusercontent.com/um-perez-alvaro/Data-Science-Practice/master/Data/papers.csv'
data = pd.read_csv(url)
data.head()

,paper,author
0,To the People of the State of New York: AFTE...,Hamilton
1,To the People of the State of New York: WHEN...,Jay
2,To the People of the State of New York: IT I...,Jay
3,To the People of the State of New York: MY L...,Jay
4,To the People of the State of New York: QUEE...,Jay


In [3]:
# Federalist paper No. 1
print(data.paper[0])

 To the People of the State of New York:  AFTER an unequivocal experience of the inefficacy of the subsisting federal government, you are called upon to deliberate on a new Constitution for the United States of America. The subject speaks its own importance; comprehending in its consequences nothing less than the existence of the UNION, the safety and welfare of the parts of which it is composed, the fate of an empire in many respects the most interesting in the world. It has been frequently remarked that it seems to have been reserved to the people of this country, by their conduct and example, to decide the important question, whether societies of men are really capable or not of establishing good government from reflection and choice, or whether they are forever destined to depend for their political constitutions on accident and force. If there be any truth in the remark, the crisis at which we are arrived may with propriety be regarded as the era in which that decision is to be ma

In [42]:
data.author.value_counts()

Hamilton            51
Madison             14
Disputed            12
Jay                  5
Hamilton+Madison     3
Name: author, dtype: int64

**Part 1 (text processing):** remove stop words and punctuations from the papers, and lemmatize them.

In [11]:
def process_pos(pos):
    if pos.startswith('J'): # adjective
        return wordnet.ADJ
    elif pos.startswith('V'): # verb
        return wordnet.VERB
    elif pos.startswith('N'): # noun
        return wordnet.NOUN
    elif pos.startswith('R'): #adverb
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()
stop_words = stopwords.words('english')
punctuation = [punc for punc in string.punctuation]

def process_text(text):
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    lemmatized_words = [lemmatizer.lemmatize(word,pos=process_pos(pos))
                       for word,pos in nltk.pos_tag(words)
                       if word not in stop_words and word not in punctuation
                       ]
    return(' '.join(lemmatized_words))

data['processed_paper'] = data.paper.apply(process_text)

**Part 2: train-test split**

We'll use the papers written by Hamilton and Madion as the training set, and the disputed papers as the testing set.

In [59]:
data.head(3)

,paper,author,processed_paper
0,To the People of the State of New York: AFTE...,Hamilton,people state new york unequivocal experience i...
1,To the People of the State of New York: WHEN...,Jay,people state new york people america reflect c...
2,To the People of the State of New York: IT I...,Jay,people state new york new observation people c...


In [51]:
data_train = data[data.author.isin(['Hamilton','Madison'])]
data_test = data[data.author=='Disputed']

Disputed    12
Name: author, dtype: int64

Extract feature matrices X_train and X_test, and target vector y_train

In [52]:
X_train = data_train.processed_paper
X_test = data_test.processed_paper
y_train = data_train.author
y_test = data_test.author

**Part 3:** build a classification pipeline (count vectorizer + Naive Bayes model) that predicts the author of a paper.

In [70]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB # or any other classifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split


In [71]:
pipe = Pipeline(steps=[
    ('vectorizer', TfidfVectorizer(max_features = 1000, ngram_range=(1,2))), # idk what ngram range is. less features better for TfidVectorizer
    ('clf', MultinomialNB())
])
pipe.fit(X_train,y_train)

Pipeline(steps=[('vectorizer',
                 TfidfVectorizer(max_features=1000, ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

**Part 4:** Use a grid search to tune the pipeline hyperparameters

In [99]:
params_dic = {'vectorizer__max_features':[500,1000,2000,4000],
             'vectorizer__ngram_range': [(1,1), (1,2), (1,3), (1,4), (1,5)],
             'vectorizer__use_idf': [False, True], # False (CountVectorizer), True (TfidfVectorizer)
              'clf__alpha': ([0.1]),
              
             }

In [100]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, params_dic, cv=5, n_jobs=-1, verbose=2)
grid.fit(X_train, y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vectorizer',
                                        TfidfVectorizer(max_features=1000,
                                                        ngram_range=(1, 2))),
                                       ('clf', MultinomialNB())]),
             n_jobs=-1,
             param_grid={'clf__alpha': [0.1],
                         'vectorizer__max_features': [500, 1000, 2000, 4000],
                         'vectorizer__ngram_range': [(1, 1), (1, 2), (1, 3),
                                                     (1, 4), (1, 5)],
                         'vectorizer__use_idf': [False, True]},
             verbose=2)

**Part 5:** How does your classification model choose between Hamilton and Madison?

In [101]:
grid.best_params_
best_pipe = grid.best_estimator_
y_test_pred = best_pipe.predict(X_test)
confusion_matrix(y_test,y_test_pred)

array([[ 0, 12],
       [ 0,  0]], dtype=int64)

In [104]:
y_test_pred

array(['Hamilton', 'Hamilton', 'Hamilton', 'Hamilton', 'Hamilton',
       'Hamilton', 'Hamilton', 'Hamilton', 'Hamilton', 'Hamilton',
       'Hamilton', 'Hamilton'], dtype='<U8')

In [41]:
# store the vocabulary
words = best_pipe['vectorizer'].get_feature_names_out()

# counts
hamilton_word_count = best_pipe['clf'].feature_count_[0,:]
madison_word_count = best_pipe['clf'].feature_count_[1,:]

words_df = pd.DataFrame({'word':words,
                        'hamilton':hamilton_word_count,
                         'madison':madison_word_count}).set_index('word')
words_df = words_df + 1
words_df

,hamilton,madison
word,,
able,0.785352,0.145650
abolish,0.159158,0.108677
absolute,0.401730,0.097894
absolutely,0.100186,0.132297
abuse,0.322754,0.040561
...,...,...
writer,0.194452,0.053840
year,0.498494,0.286139
yet,1.082627,0.302275


### Get words with highest use ratios

In [42]:
# convert counts into frequencies
words_df.hamilton = words_df.hamilton/words_df.hamilton.sum()
words_df.madison = words_df.madison/words_df.madison.sum()

# ratios
words_df['hamilton_ratio'] = words_df.hamilton/words_df.madison
words_df['madison_ratio'] = words_df.madison/words_df.hamilton

words_df.sort_values(by='madison_ratio', ascending=False).head(20)

,hamilton,madison,hamilton_ratio,madison_ratio
word,,,,
coin,0.000035,0.001044,0.033211,30.110614
legislative department,0.000046,0.001067,0.042658,23.442265
judiciary department,0.000065,0.001496,0.043133,23.183992
article confederation,0.000099,0.001797,0.055123,18.141193
consequently,0.000056,0.000971,0.058127,17.203821
executive judiciary,0.000157,0.001341,0.117333,8.522744
power government,0.000129,0.000952,0.134998,7.407511
legislative executive,0.000223,0.001559,0.142822,7.001704
old,0.000112,0.000771,0.145591,6.868554


**Part 6:** use your classifier to find who was the most likely author of the 12 disputed essays: Hamilton or Madison.

In [103]:
grid.best_params_
#best_pipe = grid.best_estimator_
#y_test_pred = best_pipe.predict(X_test)
#y_test_pred

{'clf__alpha': 0.1,
 'vectorizer__max_features': 1000,
 'vectorizer__ngram_range': (1, 1),
 'vectorizer__use_idf': True}

It predicts Hamilton wrote everything. Though other studies seem to find Madison was the author. I'm not sure why my model isn't correct. I think it might be because there's more words to train on from Hamilton than Madison.